In [1]:
import io
import os
import pandas as pd
from google.cloud import vision

### 여행 카테고리 선정

In [2]:
# 여행 카테고리 리스트
trip_category = ['accommodation', 'food', 'shopping', 'activities', 'event', 'history', 'transportation', 'weather', 'nature', 'landmark']

In [3]:
# 여행 카테고리 갯수
len(trip_category)

10

In [4]:
# API 인증을 위한 json 파일을 윈도우 환경 변수로 지정
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'c:/secret_key/My Project 45072-ed8deab854f2.json'

In [240]:
# 저장해 둔 여행지 이미지를 불러오기
path_dir = 'c:/final_project_multi/shopping'
file_list = os.listdir(path_dir)
file_name = os.path.join(path_dir, file_list[2])
with io.open(file_name, 'rb') as image_file:
    content = image_file.read()

In [242]:
# Google-cloud-vision API를 활용해 한 장의 사진에 대한 keyword 생성
client = vision.ImageAnnotatorClient()
image = vision.types.Image(content=content)
response = client.label_detection(image=image, max_results=20) # 각 이미지마다 몇개의 라벨 추출할지 결정
labels = response.label_annotations
# print(len(labels))
print(labels)

[mid: "/m/02jp1h"
description: "Marketplace"
score: 0.9276649951934814
topicality: 0.9276649951934814
, mid: "/m/0191_7"
description: "Retail"
score: 0.9194476008415222
topicality: 0.9194476008415222
, mid: "/m/0cgh4"
description: "Building"
score: 0.8933750987052917
topicality: 0.8933750987052917
, mid: "/m/02prwx"
description: "Bazaar"
score: 0.8520622849464417
topicality: 0.8520622849464417
, mid: "/m/09bgkl"
description: "Outlet store"
score: 0.8167056441307068
topicality: 0.8167056441307068
, mid: "/m/09y4pm"
description: "Market"
score: 0.7724824547767639
topicality: 0.7724824547767639
, mid: "/m/0hhdb"
description: "Shopping"
score: 0.7452040314674377
topicality: 0.7452040314674377
, mid: "/m/04f759m"
description: "Selling"
score: 0.709073007106781
topicality: 0.709073007106781
, mid: "/m/0crjs"
description: "Convenience store"
score: 0.6385955810546875
topicality: 0.6385955810546875
, mid: "/m/07br0"
description: "Trade"
score: 0.6184075474739075
topicality: 0.6184075474739075


In [245]:
# keyword List 생성
label_list = []
for label in labels:
    temp = label.description
    label_list.append(temp)

In [246]:
label_list

['Marketplace',
 'Retail',
 'Building',
 'Bazaar',
 'Outlet store',
 'Market',
 'Shopping',
 'Selling',
 'Convenience store',
 'Trade',
 'Street',
 'City']

In [247]:
# keyword Dataframe 생성
definition = pd.DataFrame({'Label': label_list})

In [248]:
definition

,Label
0,Marketplace
1,Retail
2,Building
3,Bazaar
4,Outlet store
5,Market
6,Shopping
7,Selling
8,Convenience store
9,Trade


In [251]:
# keyword 갯수
len(definition)

12

### Word2Vec 활용

In [19]:
import nltk
from gensim.models.word2vec import Word2Vec
import string

# stopword 목록 다운로드
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))

unable to import 'smart_open.gcs', disabling that module
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [93]:
# load data
with open('review_italy.txt', 'rt', encoding='utf-8') as f:
    text = f.read()
    
translator = str.maketrans('', '', string.punctuation)

In [99]:
# lonely planet review 수
cnt = 0
for i in range(len(text)):
    if text[i] == '[' or text[i] == ']':
        cnt += 1
print(cnt / 2)

19009.5


In [21]:
# review 전처리
# 1) each.translate(translator) == 특수 문자 제거
# 2) x.lower == 소문자화
# 3) if x.lower() not in stop_words == 불용어 제거
clean = [[x.lower() for x in each.translate(translator).split() if x.lower() not in stop_words] for each in text.split('.\n')]

In [23]:
# Word2Vec
# window 크기 5 (5 X 5), 최소 출현 수 2, skip-gram (1이면 사용 / 0이면 사용 X), 10번 학습 [우선 10번만 학습, 나중에 시간되면 학습 횟수 증가시키자!]
model = Word2Vec(clean, window=5, min_count=2, sg=1, iter=10)

print(list(model.wv.vocab.keys()))
print('vocab length: %d' % len(model.wv.vocab))

['hello', 'visiting', 'tuscany', '2', 'weeks', 'april', 'want', 'buy', 'nice', 'leather', 'jacket', 'slim', 'fit', 'kind', 'men', 'store', 'recommend', 'possible', 'touristy', 'overpriced', 'good', 'fine', 'locals', 'would', 'know', 'thanks', 'could', 'difficult', 'normally', 'turkish', 'chinese', 'best', 'italian', 'design', 'still', 'looking', 'try', 'san', 'lorenzo', 'market', 'florence', 'failing', 'sorts', 'small', 'shops', 'selling', 'everything', 'back', 'streets', 'expect', 'sometimes', 'workshop', 'attached', 'lots', 'local', 'stuffin', 'abovebut', 'also', 'smaller', 'citieslike', 'example', 'careful', 'travel', 'lake', 'como', 'italy', 'le', 'france', 'direct', 'route', 'via', 'car', 'anyone', 'experience', 'hiring', 'dropping', 'insight', 'appreciated', 'charge', 'lot', 'different', 'companies', 'fees', 'pay', 'minimum', '€200', 'extra', 'probably', 'conservative', 'estimate', 'usually', '€500', 'surcharge', 'renting', 'one', 'country', 'another', 'it’s', 'better', 'wait', '

In [215]:
# category가 word 사전에 있는지 확인
for i in range(len(trip_category)):
    if trip_category[i] in list(model.wv.vocab):
        print('have')
    else:
        print('not have')

have
have
have
have
have
have
have
have
have
have


#### EDA 작업 1

In [139]:
# keyword가 단어 사전에 있는지 확인하는 작업
if definition_low in list(model.wv.vocab):
    print('have')
else:
    print('not have')

not have


In [252]:
# 단어 사전에 있는 keyword 갯수, 비율
cnt = 0
not_have_list = []
for i in range(len(definition['Label'])):
    definition_low = definition['Label'][i].lower()
    # keyword가 단어 사전에 있는지 확인, 있으면 카운팅
    if definition_low in list(model.wv.vocab):
        cnt += 1
    else:
        # 말뭉치에 없는 단어 리스트 생성
        not_have_list.append(definition_low)
    
print(cnt)
have_rate = cnt / len(definition['Label'])
print(round(have_rate,2))

10
0.83


In [253]:
# 단어 사전에 없는 keyword
not_have_list

['outlet store', 'convenience store']

In [219]:
# word1과 word2의 유사도 측정
# word1이나 word2 중 하나라도 단어 사전에 없으면, error message 생성
model.wv.similarity('junk food', 'food')

KeyError: "word 'junk food' not in vocabulary"

#### EDA 작업 2

In [220]:
# 1개 사진에서 10개의 사진으로 확장해서 테스트
cnt_list = []
definition_list = []
definition_label_list = []
definition_name_list = []
have_rate_list = []
for i in range(len(file_list)):
    
    # 파일 불러오기
    file_name = os.path.join(path_dir, file_list[i])
    with io.open(file_name, 'rb') as image_file:
        content = image_file.read()
    
    # Object Detection
    client = vision.ImageAnnotatorClient()
    image = vision.types.Image(content=content)
    response = client.label_detection(image=image, max_results=20) # 각 이미지마다 몇개의 라벨 추출할지 결정
    labels = response.label_annotations

    label_list = []
    for label in labels:
        temp = label.description
        label_list.append(temp)

    definition = pd.DataFrame({'Label': label_list})
    
    # 단어 사전에 있는 keyword 갯수, 비율 구하기
    cnt = 0
    for j in range(len(definition['Label'])):
        definition_low = definition['Label'][j].lower()
        definition_name_list.append(definition_low)
        if definition_low in list(model.wv.vocab):
            cnt += 1
            definition_list.append(definition_low)
        else:
            continue
    cnt_list.append(cnt)
    definition_label_list.append(len(definition['Label']))
    have_rate = cnt / len(definition['Label'])
    have_rate_list.append(round(have_rate,2))

In [31]:
cnt_list # 임베딩 과정 거쳐서 만들어진 단어 사전에 있는 이미지별 keyword 수

[12, 10, 10, 13, 12, 9, 8, 16, 16, 13]

In [32]:
definition_label_list # 이미지별 keyword 수 (max 값: 20)

[18, 20, 20, 20, 20, 17, 20, 20, 20, 20]

In [57]:
# google-vision-api keyword와 lonely planet 단어 사전 겹치는 비율
rate = len(set(definition_list)) / len(set(definition_name_list)) # set 함수를 사용해 중복 제거
print(round(rate, 2))

0.64


In [33]:
have_rate_list # 비율

[0.67, 0.5, 0.5, 0.65, 0.6, 0.53, 0.4, 0.8, 0.8, 0.65]

In [255]:
definition['Label'][0].lower()

'marketplace'

In [256]:
# 첫 번째 사진의 keyword들과 카테고리 첫 번째인 'accomodation'과 유사도 각각 비교 
similarity_list = []
for j in range(len(definition['Label'])):
        definition_low_2 = definition['Label'][j].lower()
        if definition_low_2 in list(model.wv.vocab):
            similarity_list.append(model.wv.similarity(definition_low_2, trip_category[0]))
        else:
            continue

In [257]:
# 첫 번째 사진의 keyword들과 카테고리 첫 번째인 'accomodation'과 유사도 값
similarity_list

[0.02725341,
 0.6423179,
 0.5644326,
 0.6390025,
 0.6100374,
 0.5884269,
 0.6702323,
 0.6764588,
 0.5786922,
 0.51656246]

In [258]:
# 첫 번째 사진과 10개 카테고리 간에 각각 비교 -> 가장 높은 수치 기록하는 카테고리에 이미지 대응
similarity_list = []
match_degree = 0
image_category_match = []
for i in range(len(trip_category)):
    similarity = []
    # 한 사진의 keyword 각각과 여행 카테고리 하나와의 유사도 비교
    for j in range(len(definition['Label'])):
        definition_low_2 = definition['Label'][j].lower()
        if definition_low_2 in list(model.wv.vocab):
            similarity.append(model.wv.similarity(definition_low_2, trip_category[i]))
            match_degree += model.wv.similarity(definition_low_2, trip_category[i])
        else:
            continue
    similarity_list.append(similarity)
    image_category_match.append(match_degree)
    match_degree = 0

In [259]:
print(len(definition['Label'])) # 첫 번째 사진의 keyword 수
print(len(trip_category)) # 여행 category 수

12
10


In [260]:
similarity_list # 한 사진의 keyword 별로 카테고리 와의 유사도 수치 (첫 번째 사진의 keyword 12개, category 10개 -> 따라서, 가지의 수: 12 X 10)

[[0.02725341,
  0.6423179,
  0.5644326,
  0.6390025,
  0.6100374,
  0.5884269,
  0.6702323,
  0.6764588,
  0.5786922,
  0.51656246],
 [0.062646374,
  0.60591966,
  0.5340668,
  0.5960428,
  0.748072,
  0.76842654,
  0.703247,
  0.64587605,
  0.62372965,
  0.50501275],
 [-0.030771088,
  0.6956494,
  0.7392411,
  0.6502286,
  0.8565368,
  1.0,
  0.76239586,
  0.750678,
  0.7150825,
  0.64964175],
 [-0.08841228,
  0.66294515,
  0.64358455,
  0.6469597,
  0.6386651,
  0.78407013,
  0.6497867,
  0.7589309,
  0.50803596,
  0.5442934],
 [-0.09912665,
  0.76793885,
  0.7872885,
  0.72428083,
  0.74816155,
  0.82356244,
  0.8144833,
  0.79178905,
  0.7077804,
  0.6692992],
 [-0.036030658,
  0.65523684,
  0.63295734,
  0.56268734,
  0.6436485,
  0.7717618,
  0.6289322,
  0.68452644,
  0.5786613,
  0.54040307],
 [0.01564175,
  0.63926387,
  0.5279596,
  0.59295094,
  0.5154804,
  0.6276275,
  0.6549345,
  0.70350707,
  0.45656583,
  0.52172124],
 [-0.02745156,
  0.58809376,
  0.4353583,
  0.55110

In [261]:
# keyword들과 하나의 카테고리에 대해 나온 유사도 수치를 다 더한 값 
# (min: -12, max: 12 / keyword 갯수가 12이기 때문에) 
image_category_match

[5.513416429981589,
 5.7930396646261215,
 6.788682986050844,
 5.748859353363514,
 6.735457465052605,
 5.662784151732922,
 5.2556526977568865,
 4.487468168139458,
 5.62725580111146,
 6.8204961605370045]

In [ ]:
# -12부터 12까지의 수평선에서 위에서 구한 값(5~6)들이 어느 정도에 있는지 계산하는 함수
# 원하는 결과값을 얻기 위해 만든 자체 함수
def s(x):
    return (x + 12) / 24

In [266]:
# 2번 카테고리가 78.3% 비율로 제일 높은 수치 기록 -> 이 카테고리로 할당한다
pd.Series(image_category_match).map(s)

0    0.729726
1    0.741377
2    0.782862
3    0.739536
4    0.780644
5    0.735949
6    0.718986
7    0.686978
8    0.734469
9    0.784187
dtype: float64

In [263]:
# import numpy as np
# # sigmoid 함수
# def sigmoid(x):
#     return 1 / (1 + np.exp(-x))

In [264]:
# # softmax 함수
# def softmax(x):
#     """Compute softmax values for each sets of scores in x."""
#     e_x = np.exp(x - np.max(x))
#     return e_x / e_x.sum()

In [265]:
# # Min-Max Normalization 함수
# from sklearn.preprocessing import MinMaxScaler
# min_max_scaler = MinMaxScaler()
# fit_transformed = min_max_scaler.fit_transform(np.array(image_category_match).reshape(-1,1))
# print(fit_transformed)

[[0.43974966]
 [0.55960387]
 [0.986364  ]
 [0.54066697]
 [0.96355007]
 [0.50377277]
 [0.32926503]
 [0.        ]
 [0.48854435]
 [1.        ]]


In [267]:
# max 값 구하기
max = 0
for i in range(len(image_category_match)):
    if image_category_match[i] > max:
        max = image_category_match[i]
        max_i = i
    else:
        pass
print(max_i) # index 활용
print(trip_category[max_i]) # input photo에 대한 trip_category 대응 최종 결과

9
landmark


In [ ]:
### 개선사항
# 1. lonely planet에서 대륙별로 가장 많이 여행하는 국가에 대해서 review 추가 크롤링
# 2. trip_category 보완 -> 최종 결정
# 3. matplotlib이나 seaborn 라이브러리를 활용해 그래프로 시각화할 수 있는 부분 고민해보기! (시간 여유있으면 할 것)